# Twitter analysis for social unrest


In [165]:
import numpy as np
import pandas as pd

#Displaying data as dataframe
#pd.read_csv reads a comma-separated values (csv) file into dataframe.
df = pd.read_csv("data/dummydata.csv")

#Displaying certain columns from the dataframe to display - 
# df.loc accesses a group of rows and columns by label(s) from dataframe.
df = df.loc[0:10000,['date','user_name', 'text', 'likes', 'retweets', 'location_name']]

#Output the dataframe 
df

,date,user_name,text,likes,retweets,location_name
0,2021-01-06,nick stripe,we can adjust by population to get crude exces...,255,108,NaN
1,2021-01-06,nick stripe,we can adjust by population to get crude exces...,255,108,NaN
2,2021-01-06,femi,turning to labour or remainers like what would...,877,116,NaN
3,2021-01-06,nick stripe,numbers of deaths are affected by population s...,213,75,NaN
4,2021-01-06,NaN,here is a list of governors who preside over s...,0,7612,NaN
...,...,...,...,...,...,...
9996,2021-01-06,NaN,people think recalling gavin newsom is the sol...,0,83,NaN
9997,2021-01-06,NaN,sorry o seems you dont know im a travel consul...,8,2,NaN
9998,2021-01-06,NaN,i propose we swap all forms of travel with lau...,0,67,NaN
9999,2021-01-06,NaN,ive started my travel lets gooooo,0,0,NaN


### Preprocessing the tweets

#### Dropping duplicate tweets

In [166]:
# Removing duplicates from the dataframe
# pd.DataFrame.drop_duplicates - returns DataFrame with duplicate rows removed.
print('Dataframe size before removing duplicates', df.shape)
df = pd.DataFrame.drop_duplicates(df)
print('Dataframe size after removing duplicates', df.shape)

#Output the dataframe
df

Dataframe size before removing duplicates (10001, 6)
Dataframe size after removing duplicates (6994, 6)


,date,user_name,text,likes,retweets,location_name
0,2021-01-06,nick stripe,we can adjust by population to get crude exces...,255,108,NaN
2,2021-01-06,femi,turning to labour or remainers like what would...,877,116,NaN
3,2021-01-06,nick stripe,numbers of deaths are affected by population s...,213,75,NaN
4,2021-01-06,NaN,here is a list of governors who preside over s...,0,7612,NaN
6,2021-01-06,NaN,convince me with facts on death rates and infe...,2,0,NaN
...,...,...,...,...,...,...
9992,2021-01-06,NaN,why is your candidate ramandeep brar from bra...,13,11,NaN
9995,2021-01-06,NaN,it takes 20 years for light to travel from one...,0,90,NaN
9997,2021-01-06,NaN,sorry o seems you dont know im a travel consul...,8,2,NaN
9999,2021-01-06,NaN,ive started my travel lets gooooo,0,0,NaN
